# 📊 Селектор ликвидных фьючерсов MOEX

Этот ноутбук помогает:
1. Загрузить всю вселенную фьючерсов MOEX за указанный период
2. Загрузить фьючерсы на акции (с фильтрацией по слову "публичное" в `emitent_title`)
3. Отсортировать по ликвидности
4. Вручную отобрать нужные инструменты через чек-боксы
5. Сохранить в `portfolio_whitelist.txt`


## 1. Импорты и настройка путей


In [1]:
import time
from typing import Dict, Any, List
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display
import sys
from pathlib import Path

# Настройка путей проекта
PROJECT_ROOT = Path.cwd().resolve()
while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / "src" / "tvr_service").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

if not (PROJECT_ROOT / "src" / "tvr_service").exists():
    raise RuntimeError("Не удалось найти каталог src/tvr_service")

SRC_DIR = PROJECT_ROOT / "src"
sys.path.insert(0, str(SRC_DIR))

from tvr_service.pipeline import get_top_futures

DEFAULT_WHITELIST_PATH = PROJECT_ROOT / 'src' / 'tvr_service' / 'pipeline' / 'data' / 'portfolio_whitelist.txt'

print(f"✅ Проект: {PROJECT_ROOT}")
print(f"✅ Whitelist: {DEFAULT_WHITELIST_PATH}")


✅ Проект: C:\Users\user\Documents\piranha\constructor_TVR
✅ Whitelist: C:\Users\user\Documents\piranha\constructor_TVR\src\tvr_service\pipeline\data\portfolio_whitelist.txt


## 2. Вспомогательные функции для работы с ISS API


In [2]:
BASE_URL = "https://iss.moex.com"
SECURITIES_ENDPOINT = "/iss/securities.json"
HEADERS = {"User-Agent": "Mozilla/5.0 (futures-selector-notebook)"}
PAGE_LIMIT = 100
RETRIES = 10
RETRY_SLEEP = 1.0
REQUEST_SLEEP = 0.2


def _get_json(path: str, params: Dict[str, Any]) -> Dict[str, Any]:
    """GET запрос с ретраями."""
    url = BASE_URL + path
    last_exc = None
    for _ in range(RETRIES):
        try:
            r = requests.get(url, params=params, headers=HEADERS, timeout=30)
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_exc = e
            time.sleep(RETRY_SLEEP)
    raise RuntimeError(f"ISS request failed after {RETRIES} retries: {url} params={params}\n{last_exc}")


def _section_to_df(js: Dict[str, Any], key: str) -> pd.DataFrame:
    """Конвертация секции ISS в DataFrame."""
    sec = js.get(key, {})
    if not sec or not sec.get("data"):
        return pd.DataFrame()
    return pd.DataFrame(sec["data"], columns=sec["columns"])


def fetch_securities_table(
    table_name: str,
    q: str = None,
    lang: str = "ru",
    engine: str = None,
    market: str = None,
    is_trading: int = None,
    group_by: str = None,
    group_by_filter: str = None,
    limit: int = PAGE_LIMIT,
) -> pd.DataFrame:
    """Загрузка таблицы securities с пагинацией."""
    base_params: Dict[str, Any] = {"lang": lang, "limit": limit}
    
    if q:
        base_params["q"] = q
    if engine:
        base_params["engine"] = engine
    if market:
        base_params["market"] = market
    if is_trading is not None:
        base_params["is_trading"] = is_trading
    if group_by:
        base_params["group_by"] = group_by
    if group_by_filter:
        base_params["group_by_filter"] = group_by_filter
    
    frames: List[pd.DataFrame] = []
    start = 0
    
    while True:
        params = dict(base_params)
        params["start"] = start
        
        js = _get_json(SECURITIES_ENDPOINT, params)
        df = _section_to_df(js, table_name)
        
        if df.empty:
            break
        
        frames.append(df)
        
        if len(df) < limit:
            break
        
        start += limit
        time.sleep(REQUEST_SLEEP)
    
    if not frames:
        return pd.DataFrame()
    
    return pd.concat(frames, ignore_index=True)


def fetch_stock_futures_by_emitent(filter_word: str = "публичное") -> pd.DataFrame:
    """Загружает фьючерсы на акции, фильтруя по emitent_title."""
    print(f"📥 Загружаю таблицу 'securities' для фьючерсов...")
    
    df_securities = fetch_securities_table(
        table_name="securities",
        engine="futures",
        market="forts",
        is_trading=1,  # Только торгуемые контракты!
        lang="ru",
        limit=PAGE_LIMIT,
    )
    
    print(f"📊 Всего записей в securities (только торгуемые): {len(df_securities)}")
    
    if df_securities.empty:
        return pd.DataFrame()
    
    # Фильтрация по emitent_title
    if 'emitent_title' in df_securities.columns:
        mask = df_securities['emitent_title'].str.contains(filter_word, case=False, na=False)
        df_filtered = df_securities[mask].copy()
        print(f"🔍 После фильтрации по '{filter_word}': {len(df_filtered)} строк")
    else:
        print("⚠️ Колонка 'emitent_title' не найдена, фильтрация пропущена")
        df_filtered = df_securities.copy()
    
    # Обработка shortname для получения базового кода
    if 'shortname' in df_filtered.columns:
        df_filtered['base_code'] = df_filtered['shortname'].str.split('-').str[0].str.strip().str.upper()
        
        # Группируем по base_code, берем первую запись
        df_filtered = df_filtered.groupby('base_code').first().reset_index()
        print(f"📊 После группировки по base_code: {len(df_filtered)} строк")
    
    return df_filtered


print("✅ Вспомогательные функции загружены")


✅ Вспомогательные функции загружены


## 3. Параметры выборки

Здесь можно задать:
- **Дату или диапазон дат** для расчета ликвидности
- **Параметры сортировки** (VALUE, VOLUME, NUMTRADES)
- **Фильтр для фьючерсов на акции** (по слову в emitent_title)


In [3]:
# === Параметры даты ===
# Вариант 1: Одна дата
DATE = None  # '2025-09-25'

# Вариант 2: Диапазон дат (для более точной оценки ликвидности)
START_DATE = '2025-09-29'  # Начальная дата
END_DATE = '2025-10-03'    # Конечная дата

# === Параметры фильтрации и сортировки ===
SORT_BY = 'VALUE'          # Поле для сортировки: VALUE / VOLUME / NUMTRADES
TOP_N = None               # Сколько инструментов оставить (None = все)
FETCH_LIMIT = 100         # Лимит строк при загрузке истории

# === Фильтр для фьючерсов на акции ===
STOCK_FUTURES_FILTER_WORD = 'публичное'  # Слово для фильтрации в emitent_title

# === Пути к файлам ===
WHITELIST_PATH = DEFAULT_WHITELIST_PATH

print("✅ Параметры заданы:")
if DATE:
    print(f"   📅 Дата: {DATE}")
else:
    print(f"   📅 Диапазон: {START_DATE} — {END_DATE}")
print(f"   📊 Сортировка: {SORT_BY}")
print(f"   🔍 Фильтр акций: '{STOCK_FUTURES_FILTER_WORD}'")


✅ Параметры заданы:
   📅 Диапазон: 2025-09-29 — 2025-10-03
   📊 Сортировка: VALUE
   🔍 Фильтр акций: 'публичное'


## 4. Загрузка всей вселенной фьючерсов

Загружаем все фьючерсы без фильтров для общего обзора.


In [4]:
all_futures = get_top_futures(
    date=DATE,
    start_date=START_DATE if not DATE else None,
    end_date=END_DATE if not DATE else None,
    sort_by=SORT_BY,
    top_n=None,  # Загружаем все
    limit=FETCH_LIMIT,
    only_equities=False,  # Все фьючерсы, не только на акции
)

print(f'📊 Всего фьючерсов после агрегации: {len(all_futures)}')
all_futures[['base_code', 'VALUE', 'VOLUME', 'NUMTRADES']].head(10)


📊 Всего фьючерсов после агрегации: 412


,base_code,VALUE,VOLUME,NUMTRADES
0,GOLD,4.049981e+11,1287685.0,365474
1,SI,3.254422e+11,3864341.0,678192
2,CNY,2.951365e+11,24958033.0,595713
3,MIX,2.320085e+11,841190.0,365597
4,NG,2.111086e+11,7607301.0,757318
5,CNYRUBF,1.174774e+11,10238233.0,148598
6,IMOEXF,1.167752e+11,4355580.0,238218
7,BR,9.676048e+10,1762896.0,372102
8,SILV,9.072463e+10,2365468.0,308147
9,USDRUBF,7.758958e+10,945398.0,101801


## 5. Загрузка вселенной фьючерсов на акции (через фильтр по emitent_title)

Загружаем список фьючерсов на акции, используя фильтрацию по слову "публичное" в поле `emitent_title`.


In [6]:
stock_futures_reference = fetch_stock_futures_by_emitent(filter_word=STOCK_FUTURES_FILTER_WORD)

print(f"📊 Найдено фьючерсов на акции (по emitent_title): {len(stock_futures_reference)}")
if not stock_futures_reference.empty and 'base_code' in stock_futures_reference.columns:
    print(f"📋 Уникальных базовых кодов: {stock_futures_reference['base_code'].nunique()}")
    display(stock_futures_reference[['base_code', 'shortname', 'emitent_title']].head(10))
else:
    print("⚠️ Данные не загружены или отсутствует колонка base_code")


📥 Загружаю таблицу 'securities' для фьючерсов...
📊 Всего записей в securities (только торгуемые): 394
🔍 После фильтрации по 'публичное': 131 строк
📊 После группировки по base_code: 63 строк
📊 Найдено фьючерсов на акции (по emitent_title): 63
📋 Уникальных базовых кодов: 63


,base_code,shortname,emitent_title
0,AFKS,AFKS-3.26,"Публичное акционерное общество ""Акционерная фи..."
1,AFLT,AFLT-3.26,"Публичное акционерное общество ""Аэрофлот – рос..."
2,ALRS,ALRS-3.26,"Акционерная компания ""АЛРОСА"" (публичное акцио..."
3,ASTR,ASTR-3.26,Публичное акционерное общество Группа Астра
4,BANE,BANE-3.26,"Публичное акционерное общество ""Акционерная не..."
5,BELUGA,BELUGA-3.26,"Публичное акционерное общество ""НоваБев Групп"""
6,BSPB,BSPB-3.26,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""БАНК ""САНКТ-ПЕ..."
7,CBOM,CBOM-3.26,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе..."
8,CHMF,CHMF-3.26,"Публичное акционерное общество ""Северсталь"""
9,FEES,FEES-3.26,"Публичное акционерное общество ""Федеральная се..."


## 6. Фильтр фьючерсов по списку акций

Оставляем только те фьючерсы, которые присутствуют в списке фьючерсов на акции.


In [7]:
if not stock_futures_reference.empty and 'base_code' in stock_futures_reference.columns:
    # Получаем множество базовых кодов акций
    stock_base_codes = set(
        stock_futures_reference['base_code'].astype(str).str.upper().dropna()
    )
    
    # Фильтруем фьючерсы
    equity_futures = all_futures[
        all_futures['base_code'].astype(str).str.upper().isin(stock_base_codes)
    ].copy()
    
    print(f'📊 Фьючерсов на акции после фильтрации: {len(equity_futures)}')
else:
    equity_futures = pd.DataFrame()
    print("⚠️ Не удалось отфильтровать фьючерсы на акции")

if not equity_futures.empty:
    display(equity_futures[['base_code', 'VALUE', 'VOLUME', 'NUMTRADES']].head(10))


📊 Фьючерсов на акции после фильтрации: 63


,base_code,VALUE,VOLUME,NUMTRADES
17,SBRF,1.802603e+10,601720.0,155891
19,GAZR,1.120934e+10,915381.0,154240
23,VTBR,5.553923e+09,778110.0,75165
27,SBERF,2.860085e+09,99183.0,16599
28,LKOH,2.356577e+09,38163.0,15173
30,MVID,1.786767e+09,2954702.0,41348
32,GAZPF,1.602269e+09,135945.0,18700
35,NOTK,1.375518e+09,12240.0,7049
36,ROSN,1.234256e+09,28666.0,11753
37,YDEX,1.193815e+09,293274.0,23783


## 7. Сортировка — самые ликвидные наверху


In [8]:
# Сортируем оба датафрейма
all_futures_sorted = all_futures.sort_values(by=SORT_BY, ascending=False).reset_index(drop=True)

if not equity_futures.empty:
    equity_futures_sorted = equity_futures.sort_values(by=SORT_BY, ascending=False).reset_index(drop=True)
    print(f"✅ Отсортировано по {SORT_BY}")
    print(f"   📊 Все фьючерсы: {len(all_futures_sorted)}")
    print(f"   📊 Фьючерсы на акции: {len(equity_futures_sorted)}")
else:
    equity_futures_sorted = pd.DataFrame()
    print("⚠️ Нет данных по фьючерсам на акции для сортировки")


✅ Отсортировано по VALUE
   📊 Все фьючерсы: 412
   📊 Фьючерсы на акции: 63


## 8. Ручной отбор через чек-боксы

Выберите тип фьючерсов для отбора:
- **Все фьючерсы** — включая товарные, валютные и т.д.
- **Только фьючерсы на акции** — отфильтрованные по emitent_title

Измените переменную `USE_EQUITY_ONLY` ниже.


In [9]:
# Выберите, какие фьючерсы использовать для чек-боксов
USE_EQUITY_ONLY = True  # True = только акции, False = все фьючерсы

if USE_EQUITY_ONLY:
    futures_for_selection = equity_futures_sorted
    print(f"📋 Используем ТОЛЬКО фьючерсы на акции: {len(futures_for_selection)} инструментов")
else:
    futures_for_selection = all_futures_sorted
    print(f"📋 Используем ВСЕ фьючерсы: {len(futures_for_selection)} инструментов")

if futures_for_selection.empty:
    print("⚠️ Нет данных для отображения чек-боксов")
else:
    # Создаем чек-боксы
    checkboxes = []
    for idx, row in futures_for_selection.iterrows():
        label = f"{row['base_code']} | VALUE={row['VALUE']:.0f} | VOLUME={row['VOLUME']:.0f} | TRADES={row['NUMTRADES']}"
        cb = widgets.Checkbox(value=False, description=label, indent=False, layout=widgets.Layout(width='600px'))
        checkboxes.append(cb)
    
    checkbox_box = widgets.VBox(checkboxes)
    display(checkbox_box)
    print(f"\n✅ Отметьте нужные инструменты галочками")


📋 Используем ТОЛЬКО фьючерсы на акции: 63 инструментов



✅ Отметьте нужные инструменты галочками


## 9. Просмотр выбранных инструментов и сохранение

Используйте кнопки ниже для:
- **Предпросмотра** выбранных инструментов
- **Сохранения** в `portfolio_whitelist.txt`


In [ ]:
def _selected_indices():
    return [i for i, cb in enumerate(checkboxes) if cb.value]

preview_output = widgets.Output()
save_output = widgets.Output()

preview_button = widgets.Button(description='Показать выбранные', button_style='info')
save_button = widgets.Button(description='Сохранить в whitelist', button_style='success')

list_box = widgets.VBox([preview_button, preview_output, save_button, save_output])

def _on_preview(_):
    with preview_output:
        preview_output.clear_output()
        idxs = _selected_indices()
        if not idxs:
            print('❌ Ничего не выбрано.')
            return
        selected = futures_for_selection.iloc[idxs]
        print(f'✅ Выбрано {len(selected)} инструментов:')
        display(selected[['base_code', 'VALUE', 'VOLUME', 'NUMTRADES']])

def _on_save(_):
    with save_output:
        save_output.clear_output()
        idxs = _selected_indices()
        if not idxs:
            print('❌ Нечего сохранять — отметьте хотя бы один инструмент.')
            return
        selected_codes = futures_for_selection.iloc[idxs]['base_code'].astype(str).str.upper().tolist()
        unique_codes = sorted(set(selected_codes))
        WHITELIST_PATH.parent.mkdir(parents=True, exist_ok=True)
        WHITELIST_PATH.write_text('\n'.join(unique_codes), encoding='utf-8')
        print(f'✅ Сохранено {len(unique_codes)} тикеров в {WHITELIST_PATH}')
        print(f'📂 Файл: {WHITELIST_PATH}')
        print(f'\n📋 Список сохраненных тикеров:')
        for code in unique_codes:
            print(f'   • {code}')

preview_button.on_click(_on_preview)
save_button.on_click(_on_save)
display(list_box)
